In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

# 重新定義 LSTMModel 類 (與訓練時的定義完全相同)
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# 假設模型訓練時使用的最佳超參數為這些值
# 這些值必須與訓練時使用的超參數一致
input_size = 2      # 車速和車流量
hidden_size = 33    # 請替換為最佳的 hidden_size###############
num_layers = 3      # 請替換為最佳的 num_layers################
output_size = 1     # 預測車速

# 實例化模型
loaded_model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# 定義模型保存的路徑和檔名 (與保存時一致)
model_save_path = 'best_lstm_model.pth'

# 載入保存的模型狀態字典
loaded_model.load_state_dict(torch.load(model_save_path))
loaded_model.eval() # 將模型設定為評估模式，這會關閉 dropout 和 Batch Normalization 等訓練特有的行為
print("模型已成功載入並設定為評估模式。")

模型已成功載入並設定為評估模式。


In [ ]:
# 最新的 N 個時間步的整理好的數據 (csv檔)
new_file_path = r"D:\緯育課程\專題\TJR102_project\Data_M05A\m05a_05F0287N_05F0055N_testingDataset.csv"  #需修改為VM上面的抓取位置##############
# 確保這些數據的順序是時間遞增的，且包含 'Avg_speed' 和 'Total_volume' 兩列

new_df = pd.read_csv(new_file_path)

sequence_length = 40 #############

# 取出模型所需的 'sequence_length' 個時間步的資料
# 如果您的 sequence_length 是 40，確保 new_df 至少有 40 行數據
# 這裡假設 new_df 已經包含了足夠的歷史數據
if len(new_df) < sequence_length:
    raise ValueError(f"提供的最新數據不足 {sequence_length} 個時間步來進行預測。")

# 提取最新的 sequence_length 個時間步的特徵
# 確保 'Avg_speed' 和 'Total_volume' 的順序與訓練時一致
latest_features_np = new_df[['Avg_speed', 'Total_volume']].values[-sequence_length:]

# 將 NumPy 陣列轉換為 PyTorch 張量
# 並增加一個 batch dimension，使其形狀為 (1, sequence_length, input_size)
# 這裡的 '1' 表示一個樣本（單次預測）
input_for_prediction = torch.tensor(latest_features_np, dtype=torch.float32).unsqueeze(0)

print(f"準備好的預測輸入數據形狀: {input_for_prediction.shape}")

準備好的預測輸入數據形狀: torch.Size([1, 20, 2])


In [ ]:
# 確保模型處於評估模式 (在載入時已經設定)
# loaded_model.eval()

# 在進行預測時，禁用梯度計算，這可以節省記憶體並加速計算
with torch.no_grad():
    predicted_speed_tensor = loaded_model(input_for_prediction)

# 將預測結果從 PyTorch 張量轉換回 Python 數值
# .squeeze() 用於移除維度為 1 的單個維度 (例如 (1,1) -> 1)
# .item() 用於從單個元素張量中提取 Python 數值
predicted_average_speed = predicted_speed_tensor.squeeze().item()

print(f"預測的下一個時間步平均車速為: {predicted_average_speed:.2f}")


#將預測結果存成json
columns = ["Speed"]
data = [predicted_average_speed]

df = pd.DataFrame(data=data, columns=columns)

df.to_json('./predicted_average_speed_1.json', orient="records") #路段1的json，須根據VM上要存的位置修改前面資料夾路徑#################
# df.to_json('./predicted_average_speed_2.json', orient="records") #路段2的json，須根據VM上要存的位置修改前面資料夾路徑###############

預測的下一個時間步平均車速為: 83.03
